# Wide & Deep 模型
16年发布 用于分类和回归问题
应用到了Google Play中的应用推荐
[原始论文](https://arxiv.org/pdf/1606.07792v1.pdf)
## 稀疏特征
 * 离散值特征 性别 专业 等 只能在n个值中选则一个的信息
 * One-hot 
 * Eg:专业={计算机，人文，其他}。人文=[0,1,0] //词表
 * 叉乘 使用两个One-hot 数据进行叉乘（一一组合）
### 优点 
    有效 广泛用于工业界 广告点击预估 推荐算法
### 缺点 
    需要人工设计
    可能过拟合 所有特征都叉乘 相当于记住了每一个样本
    泛化能力差 没有出现过就不会起效果
## 密集特征
* 向量表达
    * Eg:词表={人工智能，你，他，慕课网}
    * 他=[0.3,0.2,0.6]
* Word2vec工具
### 优点
    带有语义信息 不同向量之间有相关性
    兼容没有出现过的特征组合
    更少人工参与
### 缺点
    过度泛化 推荐不怎么相关的产品
![wide wide&deep 模型对比](./2.11/截屏2021-03-02 上午11.08.33.jpg)
![wide&deep deep 模型对比](./2.11/截屏2021-03-02 上午11.09.57.jpg)
![google play 推荐算法模型图](./2.11/)

使用tf.keras 实现 Wide&Deep模型
* 子类api 
* 功能api 
* 多输入与多输出



In [1]:
import os
import sys
import time

#画图
import matplotlib as mpl 
import matplotlib.pyplot as plt
#在notebook里画图
%matplotlib inline
import numpy as np
import sklearn
#处理数据
import pandas as pd

import tensorflow as tf
from tensorflow import keras

# print(tf.__version__)
# print(sys.version_info)
# for module in mpl,np,pd,sklearn,tf,keras:
#     print(module.__name__,module.__version__)

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

# print(x_valid.shape,y_valid.shape)
# print(x_train.shape,y_train.shape)
# print(x_test.shape,y_test.shape)
# print(np.max(x_train),np.min(x_train))

In [3]:
# x = (x-u均值)/ std方差
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [4]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


In [6]:
# 函数式api
input = keras.layers.Input(shape=x_train.shape[1:])
hidden1 = keras.layers.Dense(30,activation='relu')(input)
hidden2 = keras.layers.Dense(30,activation='relu')(hidden1)

concat = keras.layers.concatenate([input,hidden2])
output = keras.layers.Dense(1)(concat)

model = keras.models.Model(inputs = [input],outputs=[output])


model.summary()

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 28, 30)       870         input_2[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 28, 30)       930         dense_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 28, 58)       0           input_2[0][0]                    
                                                                 dense_3[0][0]                

In [7]:

callbacks = [
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-2)
]

history=model.fit(x_train_scaled,y_train,epochs=100,validation_data=(x_valid_scaled,y_valid),callbacks=callbacks)

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 10s 177us/sample - loss: 3.3857 - accuracy: 0.1009 - val_loss: 3.3322 - val_accuracy: 0.0914
Epoch 2/100
55000/55000 [==============================] - 9s 158us/sample - loss: 3.3322 - accuracy: 0.1008 - val_loss: 3.3322 - val_accuracy: 0.0914
Epoch 3/100
55000/55000 [==============================] - 9s 163us/sample - loss: 3.3322 - accuracy: 0.1008 - val_loss: 3.3322 - val_accuracy: 0.0914
Epoch 4/100
55000/55000 [==============================] - 9s 162us/sample - loss: 3.3322 - accuracy: 0.1008 - val_loss: 3.3322 - val_accuracy: 0.0914
Epoch 5/100
55000/55000 [==============================] - 9s 162us/sample - loss: 3.3322 - accuracy: 0.1008 - val_loss: 3.3322 - val_accuracy: 0.0914
Epoch 6/100
55000/55000 [==============================] - 9s 161us/sample - loss: 3.3322 - accuracy: 0.1008 - val_loss: 3.3322 - val_accuracy: 0.0914


In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,3)
    plt.show()
    
plot_learning_curves(history)
#1. 参数众多 训练不充分
#2. 梯度消失 -> 链式法则 ->符合函数
#3. 批归一化 环节梯度消失

In [ ]:
model.evaluate(x_test_scaled,y_test)